In [1]:
import requests
# pretty print is used to print the output in the console in an easy to read format
from pprint import pprint
from datetime import datetime
from web3 import Web3
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import numpy as np

In [2]:
# transaction ids has to be checked in etherscan
node_url = "https://alien-sparkling-gadget.discover.quiknode.pro/ea1f5f2f5471b3a0525fa85dbfcfa27a5f90e25d/"
web3 = Web3(Web3.HTTPProvider(node_url))
tx = web3.eth.get_transaction('0xffe16a6ba847b9d189b6bb28686d1418bc1f4ef150800e4c48b5c02fe3555797')
block = tx.blockNumber
ts = web3.eth.get_block(block).timestamp
datetime.utcfromtimestamp(ts)

datetime.datetime(2023, 5, 21, 14, 41, 23)

In [3]:
from web3 import Web3
from web3 import AsyncHTTPProvider
from web3.eth import AsyncEth
import asyncio
import json
import math
import nest_asyncio
nest_asyncio.apply()


In [4]:
nodes = ['https://mainnet.infura.io/v3/174e045511c742af9cbe23f6bd053402',
         'https://mainnet.infura.io/v3/0731bcfffdb844daa0972c90e98da4e6',
         'https://mainnet.infura.io/v3/6d0c853857a249878418776d98bf732a']

In [5]:
# Define providers
w3 = Web3(Web3.HTTPProvider(nodes[0]))
providers = []
providersAsync = []
for node in nodes:
    providers.append(Web3.HTTPProvider(node))
    providersAsync.append(Web3(AsyncHTTPProvider(node), modules={"eth": (AsyncEth)}))

In [6]:
# Read factory contract addresses
# Uniswap V2 factory contract address
# contract_address = '0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f'
# SushiSwap factory contract address
# contract_address = '0xC0AEe478e3658e2610c5F7A4A2E1777cE9e4f2Ac'
with open("/home/ildefons/memeprj/Articles-mev-bot-series/Part 3/FactoriesV2.json", "r") as f:
    factories = json.load(f)

In [7]:
# Define the contract ABI
factory_abi = [
    {
        "anonymous": False,
        "inputs": [
            {
                "indexed": True,
                "internalType": "address",
                "name": "token0",
                "type": "address",
            },
            {
                "indexed": True,
                "internalType": "address",
                "name": "token1",
                "type": "address",
            },
            {
                "indexed": False,
                "internalType": "address",
                "name": "pair",
                "type": "address",
            },
            {
                "indexed": False,
                "internalType": "uint256",
                "name": "",
                "type": "uint256",
            },
        ],
        "name": "PairCreated",
        "type": "event",
    }
]

In [11]:
def getPairEvents(contract, fromBlock, toBlock):
    toBlockPrime = toBlock
    fetchCount = 0

    # Then, recursively fetch events in smaller time intervals
    def getEventsRecursive(contract, _from, _to):
        try:
            events = (
                contract.events.PairCreated()
                .create_filter(fromBlock=_from, toBlock=_to)
                .get_all_entries()
            )
            print("Found ", len(events), " events between blocks ", _from, " and ", _to)
            nonlocal fetchCount
            fetchCount += len(events)
            return events
        except ValueError:
            print("Too many events found between blocks ", _from, " and ", _to)
            midBlock = (_from + _to) // 2
            return getEventsRecursive(contract, _from, midBlock) + getEventsRecursive(
                contract, midBlock + 1, _to
            )

    return getEventsRecursive(contract, fromBlock, toBlockPrime)

In [12]:
# Fetch list of pools for each factory contract
pairDataList = []
for factoryName, factoryData in factories.items():
    events = getPairEvents(
        w3.eth.contract(address=factoryData["factory"], abi=factory_abi),
        0,
        w3.eth.block_number,
    )
    print(f"Found {len(events)} pools for {factoryName}")
    for e in events:
        pairDataList.append(
            {
                "token0": e["args"]["token0"],
                "token1": e["args"]["token1"],
                "pair": e["args"]["pair"],
                "factory": factoryName,
            }
        )

Too many events found between blocks  0  and  17749218
Found  0  events between blocks  0  and  8874609
Too many events found between blocks  8874610  and  17749218
Too many events found between blocks  8874610  and  13311914
Too many events found between blocks  8874610  and  11093262
Found  0  events between blocks  8874610  and  9983936
Too many events found between blocks  9983937  and  11093262
Found  2933  events between blocks  9983937  and  10538599
Too many events found between blocks  10538600  and  11093262
Found  5052  events between blocks  10538600  and  10815931
Found  8603  events between blocks  10815932  and  11093262
Too many events found between blocks  11093263  and  13311914
Too many events found between blocks  11093263  and  12202588
Too many events found between blocks  11093263  and  11647925
Found  8079  events between blocks  11093263  and  11370594
Found  3654  events between blocks  11370595  and  11647925
Found  4084  events between blocks  11647926  and 

In [13]:
# %%
WETH = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
pair_pool_dict = {}
for pair_object in pairDataList:
    # Check for ETH (WETH) in the pair.
    pair = (pair_object['token0'], pair_object['token1'])
    if WETH not in pair:
        continue

    # Make sure the pair is referenced in the dictionary.
    if pair not in pair_pool_dict:
        pair_pool_dict[pair] = []

    # Add the pool to the list of pools that trade this pair.
    pair_pool_dict[pair].append(pair_object)

# Create the final dictionnary of pools that will be traded on.
pool_dict = {}
for pair, pool_list in pair_pool_dict.items():
    if len(pool_list) >= 2:
        pool_dict[pair] = pool_list

In [15]:
# %%
# Helper functions for calculating the optimal trade size
# Output of a single swap
def swap_output(x, a, b, fee=0.003):
    return b * (1 - a/(a + x*(1-fee)))

# Gross profit of two successive swaps
def trade_profit(x, reserves1, reserves2, fee=0.003):
    a1, b1 = reserves1
    a2, b2 = reserves2
    return swap_output(swap_output(x, a1, b1, fee), b2, a2, fee) - x

# Optimal input amount
def optimal_trade_size(reserves1, reserves2, fee=0.003):
    a1, b1 = reserves1
    a2, b2 = reserves2
    return (math.sqrt(a1*b1*a2*b2*(1-fee)**4 * (b1*(1-fee)+b2)**2) - a1*b2*(1-fee)*(b1*(1-fee)+b2)) / ((1-fee) * (b1*(1-fee) + b2))**2

In [ ]:
# Fetch the reserves of each pool in pool_dict
to_fetch = [] # List of pool addresses for which reserves need to be fetched.
for pair, pool_list in pool_dict.items():
    for pair_object in pool_list:
        to_fetch.append(pair_object["pair"]) # Add the address of the pool
print(f"Fetching reserves of {len(to_fetch)} pools...")
# getReservesParallel() is from article 2 in the MEV bot series
reserveList = asyncio.get_event_loop().run_until_complete(getReservesParallel(to_fetch, providersAsync))